In [1]:
import json
import csv
import random

In [2]:
class Event:
    def __init__(self, cod_event, 
                 event_name, 
                 event_period, 
                 event_type, 
                 students_assigned, 
                 student_pending_counter,
                 cod_timegroup,
                 assigned_professor):
        
        self.cod_event = cod_event
        self.event_name = event_name
        self.event_period = event_period
        self.event_type = event_type
        self.students_assigned = students_assigned
        self.student_pending_counter = student_pending_counter
        self.cod_timegroup = cod_timegroup
        self.assigned_professor = assigned_professor

In [3]:
class Student:
    
    def __init__(self, name, semester, cod_pending_mandatory_events):
        self.name = name
        self.semester = semester
        self.cod_pending_mandatory_events = cod_pending_mandatory_events
        self.cod_chosen_optative_events = []

In [4]:
class Professor:
    def __init__(self, cod_professor, professor_name, cod_preferred_hours):
        self.cod_professor = cod_professor
        self.professor_name = professor_name
        self.cod_preferred_hours = cod_preferred_hours

In [5]:
class Timeslot:
    def __init__(self, cod_timewindow, cod_days, cod_time, timegroup):
        self.cod_timewindow = cod_timewindow
        self.cod_days = cod_days
        self.cod_time = cod_time
        self.timegroup = timegroup

In [59]:
class SA_Timetabling():
    
    path = "/home/leandrobataglia/Documentos/SA-Timetabling/"
    instances_path = "instances/"
    
    
    def __init__(self):
        

        self.events = []
        self.professors = []
        self.students = []
        self.timeslots = []
        self.constraints = []
        self.timetable = []
        
        self.dict_events = dict()
        self.dict_timeslots = dict()
        
        self.students_conflict_matrix = []
        self.professors_conflict_matrix = []
        timewindow_conflict_matrix = []
        
        self.students_conflict_dict = dict()
        self.professors_conflict_dict = dict()
        self.timewindow_conflict_dict = dict()
        
        self.initial_solutions = []
        
    def load_students(self, filename):
        
        f = open(type(self).path + type(self).instances_path + filename)
        data = json.load(f)
        
        for item in data:
            
            student_object = Student(item["name"],
                                      item["semester"],
                                      item["cod_pending_mandatory_events"])
            self.students.append(student_object)
        f.close()
        
        
    def load_events(self, filename):
        
        f = open(type(self).path + type(self).instances_path + filename)
        data = json.load(f)
        
        for item in data:
            event_object = Event(item["cod_event"],
                                 item["event_name"],
                                 item["event_period"],
                                 item["event_type"],
                                 item["students_assigned"],
                                 item["student_pending_counter"],
                                 item["cod_timegroup"],
                                 item["assigned_professor"])
            self.events.append(event_object)
        f.close()
        
    def load_dict_events(self, filename):
        
        dict_events = dict()
        
        f = open(type(self).path + type(self).instances_path + filename)
        data = json.load(f)
        
        for item in data:
            dict_events.update({item["cod_event"] : Event(item["cod_event"],
                                             item["event_name"],
                                             item["event_period"],
                                             item["event_type"],
                                             item["students_assigned"],
                                             item["student_pending_counter"],
                                             item["cod_timegroup"],
                                             item["assigned_professor"])})
                        
        self.dict_events = dict_events
        f.close()
        
    def load_professors(self):
        f = open(type(self).path + "TEO-professors.json")
        data = json.load(f)
        
        for item in data:
            professor_object = Professor(item["cod_professor"], 
                                         item["professor_name"], 
                                         item["cod_preferred_hours"])
            self.professors.append(professor_object)
                
        f.close()
            

        
    def load_timeslots(self):
        f = open(type(self).path + "TEO-timeslots.json")
        data = json.load(f)

        for item in data:
            timeslot_object = Timeslot(item["cod_timewindow"],
                                       item["cod_days"],
                                       item["cod_time"],
                                       item["timegroup"])                        
            self.timeslots.append(timeslot_object)
        f.close()
        
        
        
    def load_dict_timeslots(self):
        dict_timeslots = dict()
        f = open(type(self).path + "TEO-timeslots.json")
        data = json.load(f)
        for item in data:
            dict_timeslots.update({"cod_timewindow" : Timeslot(item["cod_timewindow"],
                                             item["cod_days"],
                                             item["cod_time"],
                                             item["timegroup"])})
                        
        self.dict_timeslots = dict_timeslots
        f.close()
        

        
    def create_conflict_matrices(self):
        
        self.students_conflict_matrix = []
        self.professors_conflict_matrix = []
        
        for i in self.events:
            
            ith_event_semester = []
            ith_event_professor = []
            
            for j in self.events:
                
                if (i.event_period == 0 or j.event_period == 0) and i.cod_event != j.cod_event:
                    ith_event_semester.append(0)
                elif(i.event_type == "PENDING" or j.event_type == "PENDING") and i.cod_event != j.cod_event:
                    ith_event_semester.append(0)
                elif i.event_period == j.event_period:                    
                    ith_event_semester.append(1)               
                else: ith_event_semester.append(0)
                    
                    
                if (i.assigned_professor == None or j.assigned_professor == None) and i.cod_event != j.cod_event:
                    ith_event_professor.append(0)
                elif i.assigned_professor == j.assigned_professor:
                    ith_event_professor.append(1)
                else:
                    ith_event_professor.append(0)
                    
            self.students_conflict_matrix.append(ith_event_semester)
            self.professors_conflict_matrix.append(ith_event_professor)
            
    def create_conflict_dictionaries(self):
        
        self.students_conflict_dict = dict()
        self.professors_conflict_dict = dict()
        
        for i in self.events:
            
            ith_event_semester = dict()
            ith_event_professor = dict()
            
            for j in self.events:
                
                if (i.event_period == 0 or j.event_period == 0) and i.cod_event != j.cod_event:
                    ith_event_semester.update({j.cod_event:0})
                elif(i.event_type == "PENDING" or j.event_type == "PENDING") and i.cod_event != j.cod_event:
                    ith_event_semester.update({j.cod_event:0})
                elif i.event_period == j.event_period:                    
                    ith_event_semester.update({j.cod_event:1})           
                else: 
                    ith_event_semester.update({j.cod_event:0})
                    
                    
                if (i.assigned_professor == None or j.assigned_professor == None) and i.cod_event != j.cod_event:
                    ith_event_professor.update({j.cod_event:0})
                elif i.assigned_professor == j.assigned_professor:
                    ith_event_professor.update({j.cod_event:1}) 
                else:
                    ith_event_professor.update({j.cod_event:0})
                    
            self.students_conflict_dict.update({i.cod_event:ith_event_semester})
            self.professors_conflict_dict.update({i.cod_event:ith_event_professor})
            

        
    def create_conflict_dict_event_timewindow(self):
        
        def is_event_in_timegroup(event_timegroup, timewindow_timegroups):
            if event_timegroup == -1:
                return True
            for group in timewindow_timegroups:
                if group ==  event_timegroup:
                    return True
            
            return False
                    
        self.timewindow_conflict_dict = dict()
        
        for event in self.events:
            
            ith_event = dict()
            
            for timeslot in self.timeslots:
                
                if is_event_in_timegroup(event.cod_timegroup, timeslot.timegroup):
                    ith_event.update({timeslot.cod_timewindow:1})
                else:
                    ith_event.update({timeslot.cod_timewindow:0})
                
            self.timewindow_conflict_dict.update({event.cod_event:ith_event})
            
    def create_conflict_matrix_event_timewindow(self):
        def is_event_in_timegroup(event_timegroup, timewindow_timegroups):
            if event_timegroup == -1:
                return True
            for group in timewindow_timegroups:
                if group ==  event_timegroup:
                    return True
            
            return False
                    
        self.timewindow_conflict_matrix = []
        
        for event in self.events:
            
            ith_event = []
            
            for timeslot in self.timeslots:
                
                if is_event_in_timegroup(event.cod_timegroup, timeslot.timegroup):
                    ith_event.append(1)
                else:
                    ith_event.append(0)
                
            self.timewindow_conflict_matrix.append(ith_event)
        
                
                
    def build_initial_solution(self, isprint = False):
        
        def do_events_conflict(cod_event, timeslot_list): 
            
            for timeslot_event in timeslot_list:
                if self.students_conflict_dict[cod_event][timeslot_event] == 1:
                    return False
            return True
            
        def do_professors_conflict(cod_event, timeslot_list):
            for timeslot_event in timeslot_list:
                if self.professors_conflict_dict[cod_event][timeslot_event] == 1:
                    return False
            return True
            
        
        def do_timegroups_match(cod_event, cod_timewindow):
            if self.timewindow_conflict_dict[cod_event][cod_timewindow] == 1:
                return True
            else:
                return False
        
        def print_nested_dict(dict_obj, indent = 0):
            # Iterate over all key-value pairs of dictionary
            for key, value in dict_obj.items():
                # If value is dict type, then print nested dict 
                if isinstance(value, dict):
                    print(' ' * indent, key, ':', '{')
                    print_nested_dict(value, indent + 4)
                    print(' ' * indent, '}')
                else:
                    print(' ' * indent, key, ':', value)

            
        dict_timetable = {"SEG": {"8-10":[], "10-12":[], "14-16":[], "16-18":[]},
                  "TER": {"8-10":[], "10-12":[], "14-16":[], "16-18":[]}, 
                  "QUA": {"8-10":[], "10-12":[], "14-16":[], "16-18":[]}, 
                  "QUI": {"8-10":[], "10-12":[], "14-16":[], "16-18":[]},
                  "SEX": {"8-10":[], "10-12":[], "14-16":[], "16-18":[]}}
        
        mandatory_list = [event for event in self.events if event.event_type == "MANDATORY"]
        mandatory_list = sorted(mandatory_list, key = lambda x: x.students_assigned, reverse=True)
        
        optative_list = [event for event in self.events if (event.event_type == "PENDING") or (event.event_type == "OPTIONAL")]
        optative_list = sorted(optative_list, key = lambda x: x.students_assigned + x.student_pending_counter, reverse=True)
        
        cod_event_list = []
        for event in mandatory_list:
            for i in range(2):
                cod_event_list.append(event.cod_event)
                
        
        for event in optative_list:
            for i in range(2):
                cod_event_list.append(event.cod_event)
                
        print(cod_event_list)
        
        timeslot_list = self.timeslots
        
        for cod_event in cod_event_list:
            
            flag = True
            while(flag):
                
                timeslot_index = random.randint(0, len(timeslot_list)-1)
                timeslot = timeslot_list[timeslot_index]
                
                if do_timegroups_match(cod_event, timeslot.cod_timewindow):
                    if do_professors_conflict(cod_event, dict_timetable[timeslot.cod_days][timeslot.cod_time]):
                        if do_events_conflict(cod_event, dict_timetable[timeslot.cod_days][timeslot.cod_time]):       
                            dict_timetable[timeslot.cod_days][timeslot.cod_time].append(cod_event)
                            print(cod_event + " inserted!")
                            flag = False
     
        if isprint == True:
            print_nested_dict(dict_timetable)
            
    def SA_phase1()

In [60]:
sa = SA_Timetabling()

In [61]:
sa.load_events("odd_01_events.json")

In [62]:
sa.load_dict_events("odd_01_events.json")

In [63]:
sa.load_students("odd_01_students.json")

In [64]:
sa.load_professors()

In [65]:
sa.load_timeslots()

In [66]:
sa.load_dict_timeslots()

In [67]:
sa.create_conflict_matrices()

In [ ]:
# for item in sa.students_conflict_matrix:
#     print(item)

In [36]:
# for item in sa.professors_conflict_matrix:
#     print(item)

In [73]:
# sa.create_conflict_matrix_event_timewindow()
# for item in sa.timewindow_conflict_matrix:
#     print(item)

In [69]:
sa.create_conflict_dictionaries()

In [70]:
sa.create_conflict_dict_event_timewindow()

In [17]:
for key, value in sa.dict_events.items():
        print(key, " - ", value.event_name)

IM853  -  CIRCUITOS DIGITAIS
IM885  -  GEOMETRIA ANALÍTICA
TN703  -  COMPUTADORES E SOCIEDADE
TN705  -  MATEMÁTICA DISCRETA PARA COMPUTAÇÃO
TN706  -  PROGRAMAÇÃO ESTRUTURADA
TN707  -  INTRODUÇÃO À CIÊNCIA DA COMPUTAÇÃO
TN710  -  ENGENHARIA DE SOFTWARE I
TN711  -  ESTRUTURA DE DADOS I
IM404  -  CÁLCULO II
IM478  -  ÁLGEBRA LINEAR COMPUTACIONAL
TN715  -  LINGUAGENS FORMAIS E AUTÔMATOS
TN716  -  MODELAGEM E PROJETO DE SOFTWARE
TN717  -  PROGRAMAÇÃO ORIENTADA A OBJETOS
TN718  -  GRAFOS E ALGORITMOS
IM859  -  PROBABILIDADE E ESTATÍSTICA PARA CIÊNCIA DA COMPUTAÇÃO
TN723  -  ANÁLISE E PROJETO DE ALGORITMOS
TM421  -  COMPUTAÇÃO GRÁFICA
TN724  -  INTELIGÊNCIA ARTIFICIAL
TN725  -  COMPILADORES
TN726  -  MÉTODOS NUMÉRICOS
TN727  -  BANCO DE DADOS
TN728  -  OTIMIZAÇÃO LINEAR
TM444  -  BUSCA E RECUPERAÇÃO DE INFORMAÇÃO
TM432  -  MODELAGEM DE SOFTWARE COM REDES DE PETRI
TN749  -  TEORIA DOS GRAFOS
IM188  -  CULTURA AFRO-BRASILEIRA E AFRICANA
TN734  -  ENGENHARIA DE REQUISITOS
TN740  -  TÓPICOS ESPEC

In [18]:
sa.timewindow_conflict_dict["IM853"]["SEG_8-10"]

0

In [19]:
sa.students_conflict_dict["IM853"]["IM885"]

1

In [20]:
sa.professors_conflict_dict["IM853"]["TM430"]

1

In [72]:
sa.build_initial_solution(isprint=True)

['IM853', 'IM853', 'IM885', 'IM885', 'TN703', 'TN703', 'TN705', 'TN705', 'TN706', 'TN706', 'TN707', 'TN707', 'IM404', 'IM404', 'IM478', 'IM478', 'TN715', 'TN715', 'TN716', 'TN716', 'TN717', 'TN717', 'TN718', 'TN718', 'TM421', 'TM421', 'TN724', 'TN724', 'TN725', 'TN725', 'TN726', 'TN726', 'TN727', 'TN727', 'TN728', 'TN728', 'TN710', 'TN710', 'TN711', 'TN711', 'IM859', 'IM859', 'TN723', 'TN723', 'TM444', 'TM444', 'TM432', 'TM432', 'TN749', 'TN749', 'IM188', 'IM188', 'TN734', 'TN734', 'TN740', 'TN740', 'TM430', 'TM430']
IM853 inserted!
IM853 inserted!
IM885 inserted!
IM885 inserted!
TN703 inserted!
TN703 inserted!
TN705 inserted!
TN705 inserted!
TN706 inserted!
TN706 inserted!
TN707 inserted!
TN707 inserted!
IM404 inserted!
IM404 inserted!
IM478 inserted!
IM478 inserted!
TN715 inserted!
TN715 inserted!
TN716 inserted!
TN716 inserted!
TN717 inserted!
TN717 inserted!
TN718 inserted!
TN718 inserted!
TM421 inserted!
TM421 inserted!
TN724 inserted!
TN724 inserted!
TN725 inserted!
TN725 inserte